In [274]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from tqdm import tqdm

In [275]:
theta1,theta2,theta3,theta4,theta5,theta6 = sp.symbols("theta1,theta2,theta3,theta4,theta5,theta6")
d1 = 183.3 #joint 1 height [mm]
d2 = 737.31 #joint 2 radius [mm]
d3 = 387.8 #joint 3 radius [mm]
d4 = 95.5 #joint 4 radius [mm]
d5 = 115.5 #joint 5 radius [mm]
d6 = 76.8 #end effector offset
q = sp.pi/2 #standard 90 degree angle



In [276]:
#Initialize symbols
thetai = sp.Symbol('theta_i')

t = sp.Symbol('t')
ai = sp.Symbol('a_i')
di = sp.Symbol('d_i')
alphai = sp.Symbol('alpha_i')
pi = sp.Symbol('pi')

In [277]:
thetas = [theta1,theta2,theta3,theta4,theta5,theta6]
dh_theta = [thetas[0], thetas[1]+q, thetas[2], thetas[3]+q, thetas[4], thetas[5]]
dh_a = [0, d2, d3, 0, 0, 0]
dh_d = [d1, 0, 0, d4, d5, d6]
dh_alpha = [q, 0, 0, q, -q, 0]

In [278]:
#Create dh table based on chosen coordinate frames

dh_table = {
    "": ["to 1", "to 2", "to 3", "to 4", "to 5", "to 6"],
    thetai: [dh_theta[0],dh_theta[1],dh_theta[2],dh_theta[3],dh_theta[4],dh_theta[5]],
    ai: [dh_a[0],dh_a[1],dh_a[2],dh_a[3],dh_a[4],dh_a[5]],
    di: [dh_d[0],dh_d[1],dh_d[2],dh_d[3],dh_d[4],dh_d[5]],
    alphai: [dh_alpha[0],dh_alpha[1],dh_alpha[2],dh_alpha[3],dh_alpha[4],dh_alpha[5]]

}

df = pd.DataFrame(dh_table)
df

,,theta_i,a_i,d_i,alpha_i
0,to 1,theta1,0.00,183.3,pi/2
1,to 2,theta2 + pi/2,737.31,0.0,0
2,to 3,theta3,387.80,0.0,0
3,to 4,theta4 + pi/2,0.00,95.5,pi/2
4,to 5,theta5,0.00,115.5,-pi/2
5,to 6,theta6,0.00,76.8,0


In [279]:
A_array = [sp.zeros(4,4)]*6
for i in range(0,6):
    A_array[i] = sp.Matrix([
        [sp.cos(dh_theta[i]),-sp.sin(dh_theta[i])*sp.cos(dh_alpha[i]),sp.sin(dh_theta[i])*sp.sin(dh_alpha[i]),dh_a[i]*sp.cos(dh_theta[i])],
        [sp.sin(dh_theta[i]),sp.cos(dh_theta[i])*sp.cos(dh_alpha[i]),-sp.cos(dh_theta[i])*sp.sin(dh_alpha[i]),dh_a[i]*sp.sin(dh_theta[i])],
        [0,sp.sin(dh_alpha[i]),sp.cos(dh_alpha[i]),dh_d[i]],
        [0,0,0,1]
    ])

A_final = A_array[0]
for i in range(1,6):
    A_final = A_final*A_array[i]

A_products = [A_array[0]]*6
for i in range(1,6):
    A_products[i] = A_products[i-1]*A_array[i]



In [280]:
A_array

[Matrix([
 [cos(theta1), 0,  sin(theta1),     0],
 [sin(theta1), 0, -cos(theta1),     0],
 [          0, 1,            0, 183.3],
 [          0, 0,            0,     1]]),
 Matrix([
 [-sin(theta2), -cos(theta2), 0, -737.31*sin(theta2)],
 [ cos(theta2), -sin(theta2), 0,  737.31*cos(theta2)],
 [           0,            0, 1,                   0],
 [           0,            0, 0,                   1]]),
 Matrix([
 [cos(theta3), -sin(theta3), 0, 387.8*cos(theta3)],
 [sin(theta3),  cos(theta3), 0, 387.8*sin(theta3)],
 [          0,            0, 1,                 0],
 [          0,            0, 0,                 1]]),
 Matrix([
 [-sin(theta4), 0, cos(theta4),    0],
 [ cos(theta4), 0, sin(theta4),    0],
 [           0, 1,           0, 95.5],
 [           0, 0,           0,    1]]),
 Matrix([
 [cos(theta5),  0, -sin(theta5),     0],
 [sin(theta5),  0,  cos(theta5),     0],
 [          0, -1,            0, 115.5],
 [          0,  0,            0,     1]]),
 Matrix([
 [cos(theta6), -sin(th

In [281]:
def subs_function(matrix,sub_values):
    result = matrix.subs([
        (thetas[0],sub_values[0]),
        (thetas[1],sub_values[1]),
        (thetas[2],sub_values[2]),
        (thetas[3],sub_values[3]),
        (thetas[4],sub_values[4]),
        (thetas[5],sub_values[5])     
    ])
    return result

def subs_position_vector(matrix,sub_values):
    msubs = matrix.subs([
        (thetas[0],sub_values[0]),
        (thetas[1],sub_values[1]),
        (thetas[2],sub_values[2]),
        (thetas[3],sub_values[3]),
        (thetas[4],sub_values[4]),
        (thetas[5],sub_values[5])   
    ])
    result = sp.Matrix([
        [msubs[0,3]],
        [msubs[1,3]],
        [msubs[2,3]],
        [0],
        [0],
        [0]
    ])
    return result


In [282]:


#Validate transformation matrix:

val_matrix = sp.Matrix([
    [0, q, 0, 0, 0],
    [0, 0, q, 0, 0],
    [0, 0, 0, q, 0],
    [0, 0, 0, 0, q],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0]
])

A_vals = [sp.zeros(4,4)]*5
for i in range(0,5):
    A_vals[i] = subs_function(A_final,val_matrix[:,i])


pos_vals = ["start position","first joint rotated 90 degrees","shoulder rotated 90 degrees","elbow rotated 90 degrees","wrist rotated 90 degrees"]
print("Transformation matrix validation")
for i in range(0,5):
    print("---------------")
    print(pos_vals[i])
    print(
        "x = ",((A_vals[i])[0,3]).round(2),
        "y = ",((A_vals[i])[1,3]).round(2),
        "z = ",((A_vals[i])[2,3]).round(2))




Transformation matrix validation
---------------
start position
x =  0 y =  -172.30 z =  1423.91
---------------
first joint rotated 90 degrees
x =  172.30 y =  0 z =  1423.91
---------------
shoulder rotated 90 degrees
x =  -1240.61 y =  -172.30 z =  183.30
---------------
elbow rotated 90 degrees
x =  -503.30 y =  -172.30 z =  920.61
---------------
wrist rotated 90 degrees
x =  -115.50 y =  -172.30 z =  1308.41


In [283]:
P1_ee = sp.Matrix([A_final[0,3],A_final[1,3],A_final[2,3]])


In [284]:

p_jacobian = P1_ee.jacobian([theta1,theta2,theta3,theta4,theta5,theta6])


In [285]:
z= sp.Matrix()
for transform in A_products:
  # Getting the third column from all transforms to form the Z matrix
  z=z.row_join(sp.Matrix([[transform[0,2]],[transform[1,2]],[transform[2,2]]]))


In [286]:
print("Z matrix :")
sp.pprint(z)

Z matrix :
⎡sin(θ₁)   sin(θ₁)   sin(θ₁)   (sin(θ₂)⋅sin(θ₃)⋅cos(θ₁) - cos(θ₁)⋅cos(θ₂)⋅cos( ↪
⎢                                                                              ↪
⎢-cos(θ₁)  -cos(θ₁)  -cos(θ₁)  (sin(θ₁)⋅sin(θ₂)⋅sin(θ₃) - sin(θ₁)⋅cos(θ₂)⋅cos( ↪
⎢                                                                              ↪
⎣   0         0         0                     (-sin(θ₂)⋅sin(θ₃) + cos(θ₂)⋅cos( ↪

↪ θ₃))⋅sin(θ₄) + (-sin(θ₂)⋅cos(θ₁)⋅cos(θ₃) - sin(θ₃)⋅cos(θ₁)⋅cos(θ₂))⋅cos(θ₄)  ↪
↪                                                                              ↪
↪ θ₃))⋅sin(θ₄) + (-sin(θ₁)⋅sin(θ₂)⋅cos(θ₃) - sin(θ₁)⋅sin(θ₃)⋅cos(θ₂))⋅cos(θ₄)  ↪
↪                                                                              ↪
↪ θ₃))⋅cos(θ₄) + (-sin(θ₂)⋅cos(θ₃) - sin(θ₃)⋅cos(θ₂))⋅sin(θ₄)                  ↪

↪  -((sin(θ₂)⋅sin(θ₃)⋅cos(θ₁) - cos(θ₁)⋅cos(θ₂)⋅cos(θ₃))⋅cos(θ₄) - (-sin(θ₂)⋅c ↪
↪                                                                              ↪
↪  -((sin(θ₁)⋅s

In [287]:
J = p_jacobian.col_join(z)
print(J)
J

Matrix([[-76.8*((-sin(theta1)*sin(theta2)*sin(theta3) + sin(theta1)*cos(theta2)*cos(theta3))*cos(theta4) - (sin(theta1)*sin(theta2)*cos(theta3) + sin(theta1)*sin(theta3)*cos(theta2))*sin(theta4))*sin(theta5) + 115.5*(-sin(theta1)*sin(theta2)*sin(theta3) + sin(theta1)*cos(theta2)*cos(theta3))*sin(theta4) + 115.5*(sin(theta1)*sin(theta2)*cos(theta3) + sin(theta1)*sin(theta3)*cos(theta2))*cos(theta4) + 387.8*sin(theta1)*sin(theta2)*cos(theta3) + 737.31*sin(theta1)*sin(theta2) + 387.8*sin(theta1)*sin(theta3)*cos(theta2) + 76.8*cos(theta1)*cos(theta5) + 95.5*cos(theta1), -76.8*(-(sin(theta2)*sin(theta3)*cos(theta1) - cos(theta1)*cos(theta2)*cos(theta3))*sin(theta4) + (sin(theta2)*cos(theta1)*cos(theta3) + sin(theta3)*cos(theta1)*cos(theta2))*cos(theta4))*sin(theta5) + 115.5*(sin(theta2)*sin(theta3)*cos(theta1) - cos(theta1)*cos(theta2)*cos(theta3))*cos(theta4) + 115.5*(sin(theta2)*cos(theta1)*cos(theta3) + sin(theta3)*cos(theta1)*cos(theta2))*sin(theta4) + 387.8*sin(theta2)*sin(theta3)*cos(

Matrix([
[-76.8*((-sin(theta1)*sin(theta2)*sin(theta3) + sin(theta1)*cos(theta2)*cos(theta3))*cos(theta4) - (sin(theta1)*sin(theta2)*cos(theta3) + sin(theta1)*sin(theta3)*cos(theta2))*sin(theta4))*sin(theta5) + 115.5*(-sin(theta1)*sin(theta2)*sin(theta3) + sin(theta1)*cos(theta2)*cos(theta3))*sin(theta4) + 115.5*(sin(theta1)*sin(theta2)*cos(theta3) + sin(theta1)*sin(theta3)*cos(theta2))*cos(theta4) + 387.8*sin(theta1)*sin(theta2)*cos(theta3) + 737.31*sin(theta1)*sin(theta2) + 387.8*sin(theta1)*sin(theta3)*cos(theta2) + 76.8*cos(theta1)*cos(theta5) + 95.5*cos(theta1), -76.8*(-(sin(theta2)*sin(theta3)*cos(theta1) - cos(theta1)*cos(theta2)*cos(theta3))*sin(theta4) + (sin(theta2)*cos(theta1)*cos(theta3) + sin(theta3)*cos(theta1)*cos(theta2))*cos(theta4))*sin(theta5) + 115.5*(sin(theta2)*sin(theta3)*cos(theta1) - cos(theta1)*cos(theta2)*cos(theta3))*cos(theta4) + 115.5*(sin(theta2)*cos(theta1)*cos(theta3) + sin(theta3)*cos(theta1)*cos(theta2))*sin(theta4) + 387.8*sin(theta2)*sin(theta3)*cos

In [288]:
def plot(t1,t2,t3,t4,t5,t6):
  pose = {theta1:t1,theta2:t2,theta3:t3,theta4:t4,theta5:t5,theta6:t6}
  x = [0]+[A_products[i][3].evalf(subs=pose) for i in range(len(A_products))]
  y =  [0]+[A_products[i][7].evalf(subs=pose) for i in range(len(A_products))]
  z =  [0]+[A_products[i][11].evalf(subs=pose) for i in range(len(A_products))]
  fig = plt.figure()
  k = len(A_products)
  ax = fig.add_subplot(111, projection='3d')
  ax.plot(x,y,z)
  ax.set_xlabel('x-axis')
  ax.set_ylabel('y-axis')
  ax.set_zlabel('z-axis')
  ax.set_xlim(xmin=-1000, xmax=1000)
  ax.set_ylim(ymin=-1000, ymax=1000)
  ax.set_zlim(zmin=0, zmax=1500)
  plt.show()

In [289]:
def get_velocity(inp_t,stepsize):
  T = 20
  t_ = inp_t*stepsize
  r =  50
  t1 = 8.8
  t2 = 11.6
  t3 = 17.2
  w = pi/t1
  if (t_>= 0 and t_< t1):
    vel_matrix = sp.Matrix([-w*r*sp.sin(w*t_),0,w*r*sp.cos(w*t_),0,0,0])
  elif(t_>=t1 and t_<t2):
    vel_matrix = sp.Matrix([0,0,-50/(t2-t1),0,0,0])
  elif(t_>=t2 and t_<t3):
    vel_matrix = sp.Matrix([100/(t3-t2),0,0,0,0,0])
  elif(t_>=t3 and t_<=T):
    vel_matrix = sp.Matrix([0,0,50/(T-t3),0,0,0])
  else:
    vel_matrix = sp.Matrix([0,0,0,0,0,0])
  return vel_matrix.evalf()

In [290]:
Time = 20 # We are completing the figure in 20 seconds, 1 second = 10 timestep
t1,t2,t3,t4,t5,t6 = 0,-pi/4,pi/3,pi/6,0.000000001,0.000000  # Start point or the 'S'

x = []
y = []
z = []
dis = 0
stepsize = 0.1
for t in range(int(Time/stepsize)+1):
  vel_matrix = get_velocity(t,stepsize)
  inverse_jacobian = J.evalf(subs={theta1:t1,theta2:t2,theta3:t3,theta4:t4,theta5:t5,theta6:t6}).inv()
  ang_vel = inverse_jacobian.multiply(vel_matrix)
  t1 = t1 + ang_vel[0]*stepsize
  t2 = t2 + ang_vel[1]*stepsize
  t3 = t3 + ang_vel[2]*stepsize
  t4 = t4 + ang_vel[3]*stepsize
  t5 = t5 + ang_vel[4]*stepsize
  t6 = t6 + ang_vel[5]*stepsize

  values = A_final.evalf(subs={theta1:t1,theta2:t2,theta3:t3,theta4:t4,theta5:t5,theta6:t6}).col(3)
  x.append(values[0])
  y.append(values[1])
  z.append(values[2])
plt.clf()
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x,y,z)
ax.set_xlabel('x-axis')
ax.set_ylabel('y-axis')
ax.set_zlabel('z-axis')
ax.set_ylim(ymin=-150, ymax=145)
plt.savefig("plot.png")

KeyboardInterrupt: 